In [54]:
#cargar los datos
import pandas as pd
df = pd.read_excel('requests.xlsx')
pd.set_option('display.max_columns', None)

In [ ]:
df.sample(5)

In [ ]:
#miramos los tipos de datos por columnas y los nulos.
#Nos llama la atención Request date en tipo object
df.info()

In [ ]:
df.columns

In [ ]:
df.isnull().sum()

In [ ]:
df.describe().T

In [ ]:
df.select_dtypes(include=['object']).describe().T

In [ ]:
for column in df.columns:
    print(f'{column}: {df[column].unique()}')

In [ ]:
#Comprobamos si hay reservas con el mismo id y misma cantidad
#Encontramos dos repetidas con el id 100/1200589, misma cantidad y misma fecha
#y tres con el id 100/1239393, misma cantidad y misma fecha

duplicates = df[df.duplicated(subset=['Booking', 'Amount'], keep=False)]
print(duplicates.T)

In [3]:
df = df.drop_duplicates(subset=['Booking', 'Amount'], keep='first')

In [ ]:
#comprobamos los duplicados y los borramos
df.duplicated().sum()

In [56]:
#renombramos las columnas para que no tengan espacios
name_columns = {col: col.replace(" ", "_") for col in df.columns}
df = df.rename(columns= name_columns)

In [38]:
df.sample(2)

,Booking,Request_date,Requested_by,Authorized_by,Department,Currency,Amount,Reason,Reason_2,Status,CustomerShortname,CustomerRegion,Amount_COMGES_in_EUR
193573,100/1193573,2024-07-05 00:00:00,user23@hotelbeds.com,approver25@hotelbeds.com,Sales,USD,2.02,Others,SPECIFIC CUSTOMER AGREEMENTS,Applied,CLIENT2,Region 2,1.885824
15660,100/1015660,2024-01-16 00:00:00,user23@hotelbeds.com,approver25@hotelbeds.com,Sales,USD,4.01,Others,SPECIFIC CUSTOMER AGREEMENTS,Applied,CLIENT2,Region 2,3.743640


In [5]:
#comprobamos los nulos en Athorized_by
authorized_null = df[df['Authorized_by'].isna()]

In [ ]:
print(authorized_null)

In [13]:
#vemos que los dos son de la región 2 y comprobamos si los correos de aprobación se repiten.
authorized_by_region2 = df[df['CustomerRegion'] == "Region 2"]['Authorized_by']

In [ ]:
#No hay un patrón, así que no podemos averiguar cuáles son los faltantes
print(authorized_by_region2.unique())

In [6]:
#por danto, decidimos eliminar los nulos
df = df.dropna(subset=['Authorized_by'])

In [ ]:
#Vemos que los correos deberían tener el @ antes de "hotelbeds"
df_requested = df['Requested_by']
print(df_requested)

In [ ]:
#comprobamos qué correos, tanto de requested_by como de authorized_by no tienen la estructura deseada
import re
email_pattern = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'
invalid_emails = df[~df['Requested_by'].str.contains(email_pattern, na=False) |
                            ~df['Authorized_by'].str.contains(email_pattern, na=False)]
print(invalid_emails)


In [17]:
#creamos una función para sustituir los correos erróneos y añadirles el @
import re
def change_email(row):
    email_pattern = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'
    if not re.match(email_pattern,row):
        if "hotelbeds" in row and "@" not in row:
            return row.replace("hotelbeds", "@hotelbeds")
    return row

In [18]:
df['Requested_by'] = df['Requested_by'].apply(change_email)

In [ ]:
df.columns

In [ ]:
#vemos las divisas y comprobamos que hay errores. De todas formas, vamos a sustituirlo todo por EUR y borrar la otra columna de divisa.
df['Currency'].unique()

In [11]:
#vamos a cambiar esa columna a euro con un diccionario y una función.
exchange_rates = {
    'CNY': 0.13, 'USD': 0.85, 'AED': 0.23, 'HKD': 0.11, 'GBP': 1.15, 'EUR': 1.0,
    'BRL': 0.17, 'JPY': 0.0064, 'MXN': 0.048, 'IDR': 0.000056, 'AUD': 0.63,
    'KRW': 0.00066, 'NZD': 0.59, 'ZAR': 0.049, 'INR': 0.012, 'THB': 0.025,
    'CAD': 0.63, 'COP': 0.00022, 'SAR': 0.23, 'CHF': 0.92, 'SGD': 0.63,
    'NOK': 0.085, 'PHP': 0.015
}

In [12]:
def convert_to_eur(row):
    currency = row['Currency']
    amount = row['Amount']
    if currency in exchange_rates:
        return amount * exchange_rates[currency]
    return amount

In [13]:
df['Amount'] = df.apply(convert_to_eur, axis=1)

In [ ]:
df['Currency'] = "EUR"

In [ ]:
df.sample(5)

In [42]:
#vemos cuántos nulos hay en Reason
df['Reason'].isnull().sum()

2

In [ ]:
df['Reason'].unique()

In [ ]:
#comprobamos cuántas razones hay de cada tipo
cancel_count = df['Reason'].value_counts()
print(cancel_count)

In [22]:
#agrupamos por departamento, region y cliente, por si encontramos algún patrón
grouped_by_department = df.groupby(['Department', 'Reason']).size().reset_index(name='Counts')
grouped_by_region = df.groupby(['CustomerRegion', 'Reason']).size().reset_index(name='Counts')
grouped_by_shortname = df.groupby(['CustomerShortname', 'Reason']).size().reset_index(name='Counts')

#.size cuenta el total de filas creadas por la agrupación

In [ ]:
print("Cancelaciones agrupadas por Department y Reason:")
print(grouped_by_department)


print("\nCancelaciones agrupadas por CustomerRegion y Reason:")
print(grouped_by_region)


print("\nCancelaciones agrupadas por CustomerShortname y Reason:")
print(grouped_by_shortname)

In [25]:
#mostramos solo los nulos y, efectivamente, en ambos casos el cliente y el authorized_by es el mismo
nuls = df[df['Reason'].isna()]

In [ ]:
print(nuls)

In [49]:
client113 = df[df['CustomerShortname'] == "CLIENT113"]

In [ ]:
#Revisamos que todas las entradas con CLIENT113 que tienen la razon2 "cancellation waive", tienen como reason "CANCELLATIONS" y la única que no es así, tiene reason "Booking_operational_issue". Podríamos sustituir los nulos por "CANCELLATIONS", que es la opción más probable, o eliminar esas filas o consultar con el cliente. Decidimos sustituir los nulos por "CANCELLATIONS"

print(client113)

In [15]:
df['Reason'] = df['Reason'].fillna("CANCELLATIONS")

In [104]:
df['Reason'].isnull().sum()

0

In [55]:
#De la primera razón ya no hay nulos, en la siguiente hay 82, pero son optativos.
#Aun así, los sustituimos por UNKNOWN para evitar errores.
df['Reason_2'].isnull().sum()

81

In [16]:
df['Reason_2'] = df['Reason_2'].fillna("Unknown")

In [ ]:
df['Reason_2'].unique()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [59]:
#pasamos la columna de date a formato fecha
df['Request_date'] = pd.to_datetime(df['Request_date'])

In [ ]:
nuls = df[df['Amount_COMGES_in_EUR'].isnull()]
print(nuls)

In [18]:
df = df.drop('Amount_COMGES_in_EUR', axis=1)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
#Hacemos limpieza de columnas

In [27]:
df['Reason'].unique()

array(['BOOKING_OPERATIONAL_ISSUE', 'BOOKING_TECHNICAL_ISSUE', 'OTHERS',
       'RATE_ERROR', 'CANCELLATIONS', 'Operational Issues', 'Others', nan],
      dtype=object)

In [28]:
df['Reason'] = df['Reason'].str.title()

In [29]:
df['Reason'].unique()

array(['Booking_Operational_Issue', 'Booking_Technical_Issue', 'Others',
       'Rate_Error', 'Cancellations', 'Operational Issues', nan],
      dtype=object)

In [ ]:
for column in df.columns:
    print(df[column].unique())

In [39]:
df['Reason_2'] = df['Reason_2'].str.title()

In [ ]:
df['Reason_2'].unique()

In [ ]:
#guardamos el dataframe
df.to_excel('archivo.xlsx')

In [ ]:
#A continuación, pasamos a visualización

In [41]:
import matplotlib.pyplot as plt
import seaborn as sns

In [20]:
df.columns

Index(['Booking', 'Request_date', 'Requested_by', 'Authorized_by',
       'Department', 'Currency', 'Amount', 'Reason', 'Reason_2', 'Status',
       'CustomerShortname', 'CustomerRegion'],
      dtype='object')

In [ ]:
#Distribución por tipo de cancelación

In [ ]:
import matplotlib.pyplot as plt

reason_counts = df['Reason'].value_counts()
reason_counts.plot(kind='bar', title='Distribución de tipos de cancelación')
plt.xlabel('Tipo de cancelación')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
#Distribución gasto

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))  
plt.boxplot(df['Amount'])  

plt.xlabel('Gasto')  
plt.ylabel('Frecuencia')  
plt.title('Distribución de cantidades') 
plt.show()

In [ ]:
#Euros por tipo de cancelación

In [ ]:
#cancelaciones por región

In [ ]:
plt.figure(figsize=(10, 6))
region_counts = df['CustomerRegion'].value_counts().reset_index()
region_counts.columns = ['CustomerRegion', 'Count']
sns.barplot(data=region_counts, x='CustomerRegion', y='Count', palette='viridis')
plt.xlabel('Región')
plt.ylabel('Número de cancelaciones')
plt.show()

In [ ]:
#Solicitudes en el tiempo

In [ ]:

df.groupby('Request_date').size().plot(figsize=(15,10), linestyle='-', color='teal')

plt.title("Evolución de Solicitudes a lo Largo del Tiempo")
plt.xlabel("Fecha")
plt.ylabel("Cantidad de Solicitudes")
plt.show()

In [ ]:
#correos con más peticiones

In [ ]:

correos_frecuencia = df['Requested_by'].value_counts().reset_index()
correos_frecuencia.columns = ['Requested_by', 'Count']


top_correos = correos_frecuencia.head(10)


plt.figure(figsize=(10, 6))
sns.barplot(data=top_correos, x='Requested_by', y='Count', palette='pastel')
plt.title('Correos que Más Aparecen en Requested_by')
plt.xlabel('Correo')
plt.ylabel('Número de Apariciones')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()